## SVARpy: SVAR-CUE

The core function of the SVARpy package is `SVAR.SVARest(u, estimator)`, which is used to estimate the simultaneous interactions in a non-Gaussian SVAR model. The model is represented by the equation $u_t = B_0 \epsilon_t$, where $u_t$ denotes the reduced form shocks and $\epsilon_t$ represents the structural shocks. The function supports various moment-based estimators for the estimation process.

This file provides an overview of the continuous updating version of the SVAR-GMM method implemented in Keweloh (2021) in the SVARpy package. The SVAR-CUE estimation can be performed using the following syntax:

```
SVAR.SVARest(u, estimator = 'CUE')
```
 

In [7]:
# Install required packages

# If required pip install SVARpy package
if True:
    !pip install pathos
    !pip install SVARpy

import numpy as np 
import SVAR as SVAR
np.random.seed(0)

## Simulate the SVAR

Simulate reduced form shocks $u_t$ from the SVAR

$$
u_t = B_0 ɛ_t
$$

 with $n$ variables, $T$ observations, and structural shocks $ɛ_t$ generated from a uniform distribution.

In [3]:
n = 2  # Number of variables
T = 250  # Number of observations
# Specitfy B0
B0 = np.eye(n)
# Draw structural shocks
eps = np.empty([T, n])
for i in range(n):
    eps[:, i] = np.random.uniform(low=-np.sqrt(3), high=np.sqrt(3), size=T).T
# Generate reduced form shocks u = B eps
u = np.matmul(B0, np.transpose(eps))
u = np.transpose(u)

## Estimator: SVAR-CUE

The SVAR-CUE estimator is the continuous updating version of the SVAR-GMM estimator, meaning the SVAR-CUE estimator is given by

$$
\hat{B}_T    := \underset{B \in   \mathbb{B}}{min }
g_T(B)'
W(B)
g_T(B) , 
$$

where $g_T(B)$ is analogous to the SVAR-GMM estimator and the weighting matrix $W(B) = \hat{S}(B)^{-1}$ is now continuously updated in the optimization problem.

### Run estimation:

The following script performs the estimation of the SVAR model using the SVAR-CUE estimator (`estimator='CUE'`) with the reduced form shocks $u_t$. Additional options can be specified by passing them through the prepOptions dictionary.
 

In [4]:
prepOptions = dict()
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

Estimator: CUE
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests             | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|--------------|-------------------|-------------------------------|
| T=250           | #second: 3   | WaldRec=0.87      | m=2: 1.01 / 1.0               |
| n=2             | #third: 2    | WaldRec-pval=0.35 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 3   | J=4.27            | m=4: 1.89 / 1.74              |
| #unknowns:4     | ->loss: 0.02 | J-pval=0.37       |                               |
 
|                  |       B(:,1)       |       B(:,2)       |
|------------------|--------------------|--------------------|
|      B(1,:)      |        0.97        |        0.03        |
| (avar/wald/pval) | (0.21/1125.51/0.0) |  (0.31/0.87/0.35)  |
|                  |                    |                    |
|      B(2,:)      |       -0.03        |        1.04        |
| (avar/wald/pval) |  (0.21/0.83/0.3

### Output:

The output includes the following information:

- Estimator: The estimator used in the `SVAR.SVARest(u, estimator)` function.

- Estimator Wopt/Avar: The options used to calculate the efficient weighting matrix and the asymptotic variance of the estimator.

- T: Sample size.

- n: Number of variables.

- #restrictions: Number of restrictions on the $B_0$ matrix.

- #unknowns: Number of unknown coefficients in the $B_0$ matrix.

- #second: Number of second-order moment conditions (variance and covariance conditions).

- #third: Number of third-order moment conditions (coskewness conditions).

- #fourth: Number of fourth-order moment conditions (cokurtosis conditions).

- loss: GMM loss at $\hat{B}$.

- WaldRec: Wald test statistic for the null hypothesis that the SVAR is recursive.

- WaldRec-pval: P-value corresponding to the Wald test.

- J: Test statistic of the J-Test.

- J-pval: P-value corresponding to the J-Test.

- E[$e_1^m$] with m=2: Variance $E[e_1^2]$ of all estimated shocks.

- E[$e_1^m$] with m=3: Skewness $E[e_1^3]$ of all estimated shocks.

- E[$e_1^m$] with m=4: Kurtosis $E[e_1^4]$ of all estimated shocks.


The output table displays each element of the estimated matrix $\hat{B}$. Restricted values are denoted by "x" and include the following information for each element:

- avar: Estimated asymptotic variance of the corresponding element of the matrix $\hat{B}$.

- wald: Wald test statistic for the null hypothesis that the corresponding element of the matrix $B_0$ is zero.

- pval: P-value corresponding to the Wald test.

### Options:

This section provides an overview on some available options. 

**Two-step SVAR-GMM (weighting and asymptotic variance)**

The following code shows how to pass different options for the estimator of $S = \underset{T \rightarrow \infty }{lim} E \left[T g_T(B_0) g_T(B_0)' \right] $ used to estimated the asymptotically efficient weighting matrix and to estimate the asymptotic variance of the GMM estimator.

The asymptotically efficient weighting matrix and the asymptotic variance both depend on $S = \underset{T \rightarrow \infty }{lim} E \left[T g_T(B_0) g_T(B_0)' \right] $.
However, with higher-order moment conditions, estimating $S$ can be challenging in small samples. The problem is studied in detail in Keweloh (2021b). 

With serially independent and mutually uncorrelated i.i.d shocks it holds that $S= E [f(B_0,u_t)f(B_0,u_t)'] $, see Keweloh (2021b). 
Therefore, $S$ can be estimated using $\frac{1}{T} \sum_{t=1}^{T} f(B_0,u_t)f(B_0,u_t)'$ which is implemented using the option  `'Uncorrelated_uncentered' ` 
or using the centered version $\frac{1}{T} \sum_{t=1}^{T} (f(B_0,u_t) - \bar{f})(f(B_0,u_t)- \bar{f})'  $  where $bar{f}$ is the mean of $f(B_0,u_t)$ which is implemented using the option  `'Uncorrelated' `.

With serially and mutually independent i.i.d shocks every element of the $S$ matrix can be decomposed into a sum of higher-order moments, see Keweloh (2021b). 
This option is implemented using the option  `'Independent' `.

The estimation method for $S$ can be specified seperately for the estimator of optimal weighting matrix using  `prepOptions['Wpara'] ` and for the estimator of the asymptotic variance using  `prepOptions['Avarparametric'] `.

In [5]:
# Use the assumption of serially and mutually  independent shocks to estimate the asymptotic variance (Avarparametric=Independent) 
# and the asymptotically efficent weighting matrix (Wpara=Independent), see Keweloh (2021).
prepOptions = dict()
prepOptions['Wpara'] = 'Independent'
prepOptions['Avarparametric'] = 'Independent'
SVAR_out1 = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

# Use the assumption of serially independent and mutually uncorrelated shocks to estimate the asymptotic variance (Avarparametric=Uncorrelated) 
# and the asymptotically efficent weighting matrix (Wpara=Uncorrelated), see Keweloh (2021).
prepOptions = dict()
prepOptions['Wpara'] = 'Uncorrelated'
prepOptions['Avarparametric'] = 'Uncorrelated'
SVAR_out1 = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

# Use the assumption of serially independent and mutually uncorrelated shocks to estimate the asymptotic variance (Avarparametric=Uncorrelated_uncentered) 
# and the asymptotically efficent weighting matrix (Wpara=Uncorrelated_uncentered), see Keweloh (2021).
prepOptions = dict()
prepOptions['Wpara'] = 'Uncorrelated_uncentered'
prepOptions['Avarparametric'] = 'Uncorrelated_uncentered'
SVAR_out1 = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

Estimator: CUE
Estimator Wopt/Avar: Independent/Independent
| SVAR            | Moments      | Tests            | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|--------------|------------------|-------------------------------|
| T=250           | #second: 3   | WaldRec=0.45     | m=2: 1.0 / 1.0                |
| n=2             | #third: 2    | WaldRec-pval=0.5 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 3   | J=4.35           | m=4: 1.86 / 1.74              |
| #unknowns:4     | ->loss: 0.02 | J-pval=0.36      |                               |
 
|                  |       B(:,1)       |      B(:,2)       |
|------------------|--------------------|-------------------|
|      B(1,:)      |        0.98        |       0.03        |
| (avar/wald/pval) | (0.21/1160.38/0.0) |  (0.37/0.45/0.5)  |
|                  |                    |                   |
|      B(2,:)      |       -0.02        |       1.04        |
| (avar/wald/pval) |  (0.28/0.49/0.48)  | (0.2/136

**Two-step SVAR-GMM (restrictions)**

The following code shows how to pass restrictions on the $B_0$ matrix to the estimator.

In [24]:
# Manually pass restrictions on B
restrictions = np.full([n, n], np.nan)
restrictions[0, 1] = 0  # zero restriction on position (0,1)

prepOptions = dict()
prepOptions['restrictions'] = restrictions
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

Estimator: GMM (steps=2)
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests            | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|--------------|------------------|-------------------------------|
| T=250           | #second: 3   | WaldRec=0.0      | m=2: 1.0 / 1.01               |
| n=2             | #third: 2    | WaldRec-pval=nan | m=3: -0.1 / -0.2              |
| #restrictions:1 | #fourth: 3   | J=10.78          | m=4: 1.78 / 1.86              |
| #unknowns:3     | ->loss: 0.04 | J-pval=0.06      |                               |
 
|                  |      B(:,1)       |       B(:,2)       |
|------------------|-------------------|--------------------|
|      B(1,:)      |       1.01        |       *0.0*        |
| (avar/wald/pval) | (0.2/1266.47/0.0) |   (nan/nan/nan)    |
|                  |                   |                    |
|      B(2,:)      |       -0.03       |        0.98        |
| (avar/wald/pval) | (0.34/0.87/0.35) 

**Two-step SVAR-GMM (B startvalues)**

The following code shows how to specify a start value for the B matrix used in the optimization algorithm of the GMM estimator.

In [6]:
# Start at the B matrix obtained from a recursive identification approach (default)
prepOptions = dict()
prepOptions['bstartopt'] = 'Rec'
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

# Start at the B=I
prepOptions = dict()
prepOptions['bstartopt'] = 'I'
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

# Start at the B obtained from the fast SVAR-GMM estimator in Keweloh (2021)
prepOptions = dict()
prepOptions['bstartopt'] = 'GMM_WF'
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

# Pass a specific B matrix as starting value
prepOptions = dict()
prepOptions['bstartopt'] = 'specific'
prepOptions['bstart'] = np.array(np.eye(n)[:])
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)

Estimator: CUE
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests             | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|--------------|-------------------|-------------------------------|
| T=250           | #second: 3   | WaldRec=0.87      | m=2: 1.01 / 1.0               |
| n=2             | #third: 2    | WaldRec-pval=0.35 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 3   | J=4.27            | m=4: 1.89 / 1.74              |
| #unknowns:4     | ->loss: 0.02 | J-pval=0.37       |                               |
 
|                  |       B(:,1)       |       B(:,2)       |
|------------------|--------------------|--------------------|
|      B(1,:)      |        0.97        |        0.03        |
| (avar/wald/pval) | (0.21/1125.51/0.0) |  (0.31/0.87/0.35)  |
|                  |                    |                    |
|      B(2,:)      |       -0.03        |        1.04        |
| (avar/wald/pval) |  (0.21/0.83/0.3

## References

Keweloh, Sascha Alexander. "A generalized method of moments estimator for structural vector autoregressions based on higher moments." Journal of Business & Economic Statistics 39.3 (2021): 772-782.

Keweloh, Sascha Alexander. "A feasible approach to incorporate information in higher moments in structural vector autoregressions." (2021b).